In [3]:
%reset -f
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import xarray as xr
import h5py

import matplotlib.pyplot as plt
import sys
sys.path.append("./lib")
from preprocess import *

from models import *
from GAN_class import *
from AE_class import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
#set up parameters
output_folder = "../data/preprocessed/"
file_format= "%s"
std_file_format = "np_gan_standard"
std_data_format = "np_%s"
parameters = {"nan_dim_along":"time", "nan_data_irrelevant":"absolute_height", \
              "output_folder":output_folder,"file_format":file_format, \
              "stat_dim" : "time",\
              "std_file_format":std_file_format, "std_data_format":std_data_format,\
              "std_data_list":["u","v"], \
              "std_dataset_list":["perdigao_low_res_1H_2020","perdigao_high_res_1H_2020"],\
              "num_error_tolerance":1e-5}
list_of_data_set_path=['/home/twang/data-challenge/data-challenge-2022/data/perdigao_era5_2020.nc', '/home/twang/data-challenge/data-challenge-2022/data/perdigao_low_res_1H_2020.nc', '/home/twang/data-challenge/data-challenge-2022/data/perdigao_high_res_1H_2020.nc' ]

In [3]:
#start data_preprocess
data_preprocess(list_of_data_set_path, parameters)

Creating DataSets by loading files:['/home/twang/data-challenge/data-challenge-2022/data/perdigao_era5_2020.nc', '/home/twang/data-challenge/data-challenge-2022/data/perdigao_low_res_1H_2020.nc', '/home/twang/data-challenge/data-challenge-2022/data/perdigao_high_res_1H_2020.nc']
Removing NAN indices along dimension:time
WARNING! This will change the oringal DataSets!
Searching NAN in DataSets: dict_keys(['perdigao_era5_2020', 'perdigao_low_res_1H_2020', 'perdigao_high_res_1H_2020'])...
Checking nan pattern of variable: u100  for DataSet: perdigao_era5_2020
Total number of NAN: (0,), along (0,) time indicies
Checking nan pattern of variable: v100  for DataSet: perdigao_era5_2020
Total number of NAN: (0,), along (0,) time indicies
Checking nan pattern of variable: t2m  for DataSet: perdigao_era5_2020
Total number of NAN: (0,), along (0,) time indicies
Checking nan pattern of variable: i10fg  for DataSet: perdigao_era5_2020
Total number of NAN: (0,), along (0,) time indicies
NAN pattern a

In [105]:
parameters = dict()

parameters["train"] = {"learning_rate_g": 1e-4, 
                       "learning_rate_d": 1e-4,
                       "beta_1": 0.9,
                       "beta_2": 0.999,
                       "epsilon": 1e-08,
                       "batch_size": 128,
                       "alpha_advers": 1e-3}
parameters["data"] = {'output_folder': "../data/preprocessed/",
                      'file_format': "np_gan_standard",
                      'xy_keyword_dict': {"x":"low", "y":"high"},
                      'xy_exclude_list': ["stddev", "mean"]}



In [106]:
model = AWWSM4_SR_GAN(parameters=parameters, is_GAN=True, doing_pretrain=True)

Model: "model_28"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_29 (InputLayer)          [(None, 96, 96, 2)]  0           []                               
                                                                                                  
 conv2d_transpose_504 (Conv2DTr  (None, 96, 96, 64)  1216        ['input_29[0][0]']               
 anspose)                                                                                         
                                                                                                  
 activation_252 (Activation)    (None, 96, 96, 64)   0           ['conv2d_transpose_504[0][0]']   
                                                                                                  
 conv2d_transpose_505 (Conv2DTr  (None, 96, 96, 64)  36928       ['activation_252[0][0]']  

In [107]:
model.load_data()

Data in file ../data/preprocessed/np_gan_standard.h5 are: 
 ['np_perdigao_high_res_1H_2020_mean', 'np_perdigao_high_res_1H_2020_std', 'np_perdigao_high_res_1H_2020_stddev', 'np_perdigao_low_res_1H_2020_mean', 'np_perdigao_low_res_1H_2020_std', 'np_perdigao_low_res_1H_2020_stddev']
Examining data np_perdigao_high_res_1H_2020_mean
Examining data np_perdigao_high_res_1H_2020_std
Examining data np_perdigao_high_res_1H_2020_stddev
Examining data np_perdigao_low_res_1H_2020_mean
Examining data np_perdigao_low_res_1H_2020_std
Loading data np_perdigao_low_res_1H_2020_std from file ../data/preprocessed/np_gan_standard.h5
Data in file ../data/preprocessed/np_gan_standard.h5 are: 
 ['np_perdigao_high_res_1H_2020_mean', 'np_perdigao_high_res_1H_2020_std', 'np_perdigao_high_res_1H_2020_stddev', 'np_perdigao_low_res_1H_2020_mean', 'np_perdigao_low_res_1H_2020_std', 'np_perdigao_low_res_1H_2020_stddev']
Examining data np_perdigao_high_res_1H_2020_mean
Examining data np_perdigao_high_res_1H_2020_std
L

In [108]:
model.split_data()

x_train.shape: (5112, 96, 96, 2)
x_val.shape: (1704, 96, 96, 2)
x_test.shape: (1704, 96, 96, 2)
y_train.shape: (5112, 192, 192, 2)
y_val.shape: (1704, 192, 192, 2)
y_test.shape: (1704, 192, 192, 2)
6.890482 5.989441 5.7024393 -5.4777956 -5.0524907 -5.5057893
7.035497 5.8600492 6.0442076 -5.315754 -5.2317853 -5.2996855


In [34]:
model.pretrain()

Training network ...
Epoch: 1
Epoch generator training loss = 0.046698, val loss = 0.042000
Epoch took 45.90 seconds

Epoch: 2
Epoch generator training loss = 0.038540, val loss = 0.035931
Epoch took 45.62 seconds

Epoch: 3


2022-08-23 23:52:28.851513: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.61GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-08-23 23:52:30.970847: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.61GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-08-23 23:52:33.105531: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.61GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-08-23 23:52:35.233846: W tensorflow/core/common_runtime/bfc_allocato

Epoch generator training loss = 0.033756, val loss = 0.032241
Epoch took 46.64 seconds

Epoch: 4
Epoch generator training loss = 0.030678, val loss = 0.029784
Epoch took 47.03 seconds

Epoch: 5
Epoch generator training loss = 0.028586, val loss = 0.028015
Epoch took 46.54 seconds

Epoch: 6
Epoch generator training loss = 0.027082, val loss = 0.026670
Epoch took 46.73 seconds

Epoch: 7
Epoch generator training loss = 0.025921, val loss = 0.025700
Epoch took 46.80 seconds

Epoch: 8
Epoch generator training loss = 0.024980, val loss = 0.024928
Epoch took 46.77 seconds

Epoch: 9
Epoch generator training loss = 0.024245, val loss = 0.024269
Epoch took 46.87 seconds

Epoch: 10
Epoch generator training loss = 0.023640, val loss = 0.023692
Epoch took 46.89 seconds

Epoch: 11
Epoch generator training loss = 0.023114, val loss = 0.023148
Epoch took 46.95 seconds

Epoch: 12
Epoch generator training loss = 0.022658, val loss = 0.022645
Epoch took 48.00 seconds

Epoch: 13
Epoch generator training l

In [35]:
print(model.pretrain_train_loss)

[<tf.Tensor: shape=(), dtype=float32, numpy=0.046698164>, <tf.Tensor: shape=(), dtype=float32, numpy=0.038540207>, <tf.Tensor: shape=(), dtype=float32, numpy=0.033755895>, <tf.Tensor: shape=(), dtype=float32, numpy=0.030678201>, <tf.Tensor: shape=(), dtype=float32, numpy=0.028585637>, <tf.Tensor: shape=(), dtype=float32, numpy=0.027082259>, <tf.Tensor: shape=(), dtype=float32, numpy=0.025920695>, <tf.Tensor: shape=(), dtype=float32, numpy=0.024979673>, <tf.Tensor: shape=(), dtype=float32, numpy=0.024245195>, <tf.Tensor: shape=(), dtype=float32, numpy=0.023640398>, <tf.Tensor: shape=(), dtype=float32, numpy=0.023113502>, <tf.Tensor: shape=(), dtype=float32, numpy=0.022657823>, <tf.Tensor: shape=(), dtype=float32, numpy=0.02226489>, <tf.Tensor: shape=(), dtype=float32, numpy=0.02192056>, <tf.Tensor: shape=(), dtype=float32, numpy=0.021604775>, <tf.Tensor: shape=(), dtype=float32, numpy=0.021314677>, <tf.Tensor: shape=(), dtype=float32, numpy=0.021047855>, <tf.Tensor: shape=(), dtype=floa

In [36]:
model.save_gen_model("./temp_v0_gen.h5")

In [42]:
model.pretrain(epochs=1)

Training network ...
Epoch: 1
Epoch generator training loss = 0.020239, val loss = 0.020219
Epoch took 47.32 seconds

Pretrain Done.


In [43]:
model2 = AWWSM4_SR_GAN(parameters=parameters, is_GAN=True, doing_pretrain=True)

model2.load_gen_model("./temp_v0_gen.h5")
model2.load_data()
model2.split_data()

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 96, 96, 2)]  0           []                               
                                                                                                  
 conv2d_transpose_252 (Conv2DTr  (None, 96, 96, 64)  1216        ['input_15[0][0]']               
 anspose)                                                                                         
                                                                                                  
 activation_126 (Activation)    (None, 96, 96, 64)   0           ['conv2d_transpose_252[0][0]']   
                                                                                                  
 conv2d_transpose_253 (Conv2DTr  (None, 96, 96, 64)  36928       ['activation_126[0][0]']  

In [44]:
model2.pretrain(epochs=1)

Training network ...
Epoch: 1
Epoch generator training loss = 0.059382, val loss = 0.025276
Epoch took 48.12 seconds

Pretrain Done.


In [45]:
model.save_gen_weights("./temp_v0_gen_weights.h5")

In [46]:
model3 = AWWSM4_SR_GAN(parameters=parameters, is_GAN=True, doing_pretrain=True)

model3.load_gen_weights("./temp_v0_gen_weights.h5")
model3.load_data()
model3.split_data()

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 96, 96, 2)]  0           []                               
                                                                                                  
 conv2d_transpose_252 (Conv2DTr  (None, 96, 96, 64)  1216        ['input_15[0][0]']               
 anspose)                                                                                         
                                                                                                  
 activation_126 (Activation)    (None, 96, 96, 64)   0           ['conv2d_transpose_252[0][0]']   
                                                                                                  
 conv2d_transpose_253 (Conv2DTr  (None, 96, 96, 64)  36928       ['activation_126[0][0]']  

In [47]:
model3.pretrain(epochs=1)

Training network ...
Epoch: 1
Epoch generator training loss = 0.056726, val loss = 0.024809
Epoch took 46.28 seconds

Pretrain Done.


In [48]:
model4 = AWWSM4_SR_GAN(parameters=parameters, is_GAN=True, doing_pretrain=True)

#model4.load_gen_weights("./temp_v0_gen_weights.h5")
model4.load_data()
model4.split_data()

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 96, 96, 2)]  0           []                               
                                                                                                  
 conv2d_transpose_252 (Conv2DTr  (None, 96, 96, 64)  1216        ['input_15[0][0]']               
 anspose)                                                                                         
                                                                                                  
 activation_126 (Activation)    (None, 96, 96, 64)   0           ['conv2d_transpose_252[0][0]']   
                                                                                                  
 conv2d_transpose_253 (Conv2DTr  (None, 96, 96, 64)  36928       ['activation_126[0][0]']  

In [55]:
print(model4.gen.layers[1].get_weights()[0][0,0,:,:])

[[ 9.3461111e-02  8.0226198e-02]
 [ 9.3294740e-02  5.5610400e-02]
 [-9.5951751e-02 -5.4663379e-02]
 [-9.1389455e-02  3.3974864e-02]
 [ 7.0188139e-03 -5.7225719e-02]
 [ 4.1558329e-02 -1.8053548e-02]
 [-9.0926729e-02 -8.5607380e-02]
 [ 6.6296086e-02  8.5485883e-02]
 [-4.3606799e-02 -6.7777462e-02]
 [-1.5589771e-02 -3.3594649e-02]
 [ 6.3462853e-02  3.8415354e-02]
 [ 8.6971082e-02  7.2716571e-02]
 [ 6.6878729e-02  7.9909362e-02]
 [ 3.7158180e-02  1.7722495e-02]
 [ 1.6092716e-02 -8.0275215e-02]
 [-8.4511288e-02 -5.4112165e-03]
 [-6.7846723e-02  4.0140178e-02]
 [ 7.2264582e-02 -5.3411494e-03]
 [ 2.6979959e-02  9.8261602e-02]
 [-6.9669880e-02 -7.2077148e-02]
 [ 4.6253487e-02 -3.1005850e-02]
 [ 8.1674255e-02 -9.2474699e-02]
 [-3.0707607e-02 -3.9599594e-02]
 [-4.1276696e-03  8.6525626e-02]
 [-4.5631681e-02 -9.2053488e-02]
 [-6.1020248e-02 -7.3890410e-02]
 [ 4.2782374e-02 -1.7594239e-02]
 [ 6.1676815e-02 -7.3236518e-02]
 [-9.2687279e-02  4.6715890e-05]
 [-5.9274238e-02  8.7523401e-02]
 [ 4.63019

In [56]:
model4.load_gen_weights("./temp_v0_gen_weights.h5")
print(model4.gen.layers[1].get_weights()[0][0,0,:,:])

[[ 0.09365644  0.08049852]
 [ 0.09302788  0.05556092]
 [-0.09638616 -0.05434787]
 [-0.09120344  0.0341416 ]
 [ 0.00708121 -0.05684371]
 [ 0.04177841 -0.01811223]
 [-0.09063689 -0.08592439]
 [ 0.06645311  0.08516557]
 [-0.04344662 -0.06762868]
 [-0.01576529 -0.03356375]
 [ 0.06306928  0.03875286]
 [ 0.08668686  0.07264739]
 [ 0.06600367  0.08047444]
 [ 0.03673158  0.01819682]
 [ 0.01522929 -0.08056803]
 [-0.08580772 -0.00536646]
 [-0.06778652  0.04031411]
 [ 0.07271931 -0.00518505]
 [ 0.02664844  0.09744154]
 [-0.06979138 -0.07196081]
 [ 0.04649573 -0.03115   ]
 [ 0.08177029 -0.09215222]
 [-0.03094587 -0.04005896]
 [-0.00428726  0.08662104]
 [-0.04554687 -0.09222048]
 [-0.06153265 -0.07375995]
 [ 0.04215042 -0.01711746]
 [ 0.06221461 -0.0732188 ]
 [-0.09288383  0.00024858]
 [-0.0592109   0.08799955]
 [ 0.04633901 -0.02512094]
 [-0.07425226 -0.00358385]
 [-0.00581716 -0.07384498]
 [ 0.05514409  0.05973893]
 [-0.08573145 -0.00446259]
 [-0.00647888 -0.0206461 ]
 [-0.02066275 -0.0564805 ]
 

In [58]:
print(model.gen.layers[1].get_weights()[0][0,0,1:8,:])

[[ 0.09302788  0.05556092]
 [-0.09638616 -0.05434787]
 [-0.09120344  0.0341416 ]
 [ 0.00708121 -0.05684371]
 [ 0.04177841 -0.01811223]
 [-0.09063689 -0.08592439]
 [ 0.06645311  0.08516557]]


In [59]:
model.save_gen_model("./temp_v1_gen.h5")

In [60]:
model2.load_gen_model("./temp_v1_gen.h5")

In [61]:
print(model2.gen.layers[1].get_weights()[0][0,0,1:8,:])

[[ 0.09302788  0.05556092]
 [-0.09638616 -0.05434787]
 [-0.09120344  0.0341416 ]
 [ 0.00708121 -0.05684371]
 [ 0.04177841 -0.01811223]
 [-0.09063689 -0.08592439]
 [ 0.06645311  0.08516557]]


In [62]:
model2.gen.predict(model.data_x[0:1,:,:,:])

1/1 [==============================] - 0s 385ms/step


array([[[[-0.37485835,  0.30726415],
         [-0.40808907,  0.05885644],
         [-0.50113237,  0.35064375],
         ...,
         [-0.7805464 ,  0.01353141],
         [-0.6150223 , -0.3512819 ],
         [-0.38132903,  0.06250219]],

        [[-0.52804154,  0.33338013],
         [-0.49888432,  0.19817083],
         [-0.6374717 ,  0.28345394],
         ...,
         [-0.7952736 , -0.12297333],
         [-0.65611494,  0.01302711],
         [-0.58305264,  0.14513022]],

        [[-0.65991914,  0.20227623],
         [-0.73194146,  0.20666243],
         [-0.5085219 ,  0.3249064 ],
         ...,
         [-0.74954367,  0.12282849],
         [-0.7682737 , -0.10580962],
         [-0.6848687 , -0.00848786]],

        ...,

        [[-0.6528366 , -0.15003683],
         [-0.383917  , -0.21763699],
         [-0.65715307, -0.08540032],
         ...,
         [-0.38353774,  0.05535938],
         [-0.17688234,  0.10171083],
         [-0.25343248,  0.03382051]],

        [[-0.4991667 , -0.10792432

In [63]:
model.gen.predict(model.data_x[0:1,:,:,:])

1/1 [==============================] - 0s 15ms/step


array([[[[-0.37485835,  0.30726415],
         [-0.40808907,  0.05885644],
         [-0.50113237,  0.35064378],
         ...,
         [-0.7805464 ,  0.01353141],
         [-0.6150223 , -0.35128188],
         [-0.38132903,  0.06250218]],

        [[-0.5280416 ,  0.33338013],
         [-0.4988843 ,  0.19817081],
         [-0.6374717 ,  0.28345394],
         ...,
         [-0.79527366, -0.12297333],
         [-0.6561149 ,  0.01302711],
         [-0.58305264,  0.14513022]],

        [[-0.65991914,  0.20227624],
         [-0.73194146,  0.20666243],
         [-0.5085219 ,  0.3249064 ],
         ...,
         [-0.7495437 ,  0.12282851],
         [-0.7682737 , -0.10580962],
         [-0.6848687 , -0.00848786]],

        ...,

        [[-0.6528366 , -0.15003683],
         [-0.38391697, -0.21763702],
         [-0.65715307, -0.08540032],
         ...,
         [-0.38353774,  0.05535938],
         [-0.17688234,  0.10171083],
         [-0.25343248,  0.03382051]],

        [[-0.4991667 , -0.10792433

In [65]:
print(model.data_x[0,34,12,:])

[-0.51984024  0.37347415]


In [66]:
print(model2.data_x[0,34,12,:])

[-0.51984024  0.37347415]


In [67]:
model.pretrain(epochs=1)

Training network ...
Epoch: 1
Epoch generator training loss = 0.020073, val loss = 0.020063
Epoch took 45.90 seconds

Pretrain Done.


In [68]:
model2.pretrain(epochs=1)

Training network ...
Epoch: 1
Epoch generator training loss = 0.041228, val loss = 0.022993
Epoch took 46.39 seconds

Pretrain Done.


In [69]:
print(model.x_train[0,34,12,:])

[-0.172217    0.28416803]


In [70]:
print(model2.x_train[0,34,12,:])

[-0.172217    0.28416803]


In [74]:
print(id(model2.gen))

140207065014816


In [75]:
print(id(model.gen))

140208942976208


In [76]:
model.gen.save('./temp_v2_gen.h5')

In [77]:
gen_model = tf.keras.models.load_model('./temp_v2_gen.h5')

In [78]:
model2 = AWWSM4_SR_GAN(parameters=parameters, generator=gen_model, is_GAN=True, doing_pretrain=True)

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 96, 96, 2)]  0           []                               
                                                                                                  
 conv2d_transpose_108 (Conv2DTr  (None, 96, 96, 64)  1216        ['input_7[0][0]']                
 anspose)                                                                                         
                                                                                                  
 activation_54 (Activation)     (None, 96, 96, 64)   0           ['conv2d_transpose_108[0][0]']   
                                                                                                  
 conv2d_transpose_109 (Conv2DTr  (None, 96, 96, 64)  36928       ['activation_54[0][0]']    

In [79]:
model2.load_data()
model2.split_data()

Data in file ../data/preprocessed/np_gan_standard.h5 are: 
 ['np_perdigao_high_res_1H_2020_mean', 'np_perdigao_high_res_1H_2020_std', 'np_perdigao_high_res_1H_2020_stddev', 'np_perdigao_low_res_1H_2020_mean', 'np_perdigao_low_res_1H_2020_std', 'np_perdigao_low_res_1H_2020_stddev']
Examining data np_perdigao_high_res_1H_2020_mean
Examining data np_perdigao_high_res_1H_2020_std
Examining data np_perdigao_high_res_1H_2020_stddev
Examining data np_perdigao_low_res_1H_2020_mean
Examining data np_perdigao_low_res_1H_2020_std
Loading data np_perdigao_low_res_1H_2020_std from file ../data/preprocessed/np_gan_standard.h5
Data in file ../data/preprocessed/np_gan_standard.h5 are: 
 ['np_perdigao_high_res_1H_2020_mean', 'np_perdigao_high_res_1H_2020_std', 'np_perdigao_high_res_1H_2020_stddev', 'np_perdigao_low_res_1H_2020_mean', 'np_perdigao_low_res_1H_2020_std', 'np_perdigao_low_res_1H_2020_stddev']
Examining data np_perdigao_high_res_1H_2020_mean
Examining data np_perdigao_high_res_1H_2020_std
L

In [80]:
model2.pretrain(epochs=1)

Training network ...
Epoch: 1
Epoch generator training loss = 0.053948, val loss = 0.024790
Epoch took 46.20 seconds

Pretrain Done.


In [81]:
model2.pretrain(epochs=2)

Training network ...
Epoch: 1
Epoch generator training loss = 0.022324, val loss = 0.021319
Epoch took 46.23 seconds

Epoch: 2
Epoch generator training loss = 0.020621, val loss = 0.020445
Epoch took 45.84 seconds

Pretrain Done.


In [82]:
model2.pretrain(epochs=2)

Training network ...
Epoch: 1
Epoch generator training loss = 0.020060, val loss = 0.020078
Epoch took 46.30 seconds

Epoch: 2
Epoch generator training loss = 0.019629, val loss = 0.019729
Epoch took 46.15 seconds

Pretrain Done.


In [109]:
model.load_gen_model("./temp_v1_gen.h5")

In [110]:
model.pretrain(epochs=1)

Training network ...
Epoch: 1
Epoch generator training loss = 0.056726, val loss = 0.024809
Epoch took 46.69 seconds

Pretrain Done.


In [111]:
model.doing_pretrain = False
model.train_GAN(epochs=10)

Training network ...
Epoch: 1
Using d_loss_ideal_range:[0.45, 0.65], g_reflect_max: 20, d_reflect_max: 20
Using d_loss_ideal_range:[0.45, 0.65], g_reflect_max: 20, d_reflect_max: 20
Using d_loss_ideal_range:[0.45, 0.65], g_reflect_max: 20, d_reflect_max: 20
Epoch generator loss = 0.023326, discriminator loss = 0.555556, g_count = 109, d_count = 56
Epoch val: g_loss = 0.021971, d_loss = 0.498562, content_loss = 0.020233, advers_loss = 1.738255
Epoch took 139.24 seconds

Epoch: 2
Epoch generator loss = 0.019731, discriminator loss = 0.447163, g_count = 551, d_count = 40
Epoch val: g_loss = 0.019462, d_loss = 0.428906, content_loss = 0.018021, advers_loss = 1.441274
Epoch took 533.03 seconds

Epoch: 3
Epoch generator loss = 0.018479, discriminator loss = 0.448251, g_count = 537, d_count = 41
Epoch val: g_loss = 0.018634, d_loss = 0.424944, content_loss = 0.017098, advers_loss = 1.536617
Epoch took 520.63 seconds

Epoch: 4


KeyboardInterrupt: 

In [29]:
parameters = dict()

parameters["train"] = {"learning_rate_g": 1e-4, 
                       "learning_rate_d": 1e-4,
                       "beta_1": 0.9,
                       "beta_2": 0.999,
                       "epsilon": 1e-08,
                       "batch_size": 128,
                       "shuffle": True,
                        "n_sub_net":4,
                       "latent_dim_en":18,
                        "latent_dim_de_origin":18,
                        "log_format":"logggggg",
                          "epochs":2}
parameters["data"] = {'output_folder': "../data/preprocessed/",
                      'file_format': "np_gan_standard",
                      'xy_keyword_dict': {"x":"low", "y":"high"},
                      'xy_exclude_list': ["stddev", "mean"]}

In [30]:
model_AE = AWWSM4_HIER_AE(parameters)

In [31]:
model_AE.load_data()

Data in file ../data/preprocessed/np_gan_standard.h5 are: 
 ['np_perdigao_high_res_1H_2020_mean', 'np_perdigao_high_res_1H_2020_std', 'np_perdigao_high_res_1H_2020_stddev', 'np_perdigao_low_res_1H_2020_mean', 'np_perdigao_low_res_1H_2020_std', 'np_perdigao_low_res_1H_2020_stddev']
Examining data np_perdigao_high_res_1H_2020_mean
Examining data np_perdigao_high_res_1H_2020_std
Examining data np_perdigao_high_res_1H_2020_stddev
Examining data np_perdigao_low_res_1H_2020_mean
Examining data np_perdigao_low_res_1H_2020_std
Loading data np_perdigao_low_res_1H_2020_std from file ../data/preprocessed/np_gan_standard.h5
Data in file ../data/preprocessed/np_gan_standard.h5 are: 
 ['np_perdigao_high_res_1H_2020_mean', 'np_perdigao_high_res_1H_2020_std', 'np_perdigao_high_res_1H_2020_stddev', 'np_perdigao_low_res_1H_2020_mean', 'np_perdigao_low_res_1H_2020_std', 'np_perdigao_low_res_1H_2020_stddev']
Examining data np_perdigao_high_res_1H_2020_mean
Examining data np_perdigao_high_res_1H_2020_std
L

In [37]:
model_AE.split_data()

X_train.shape: (5112, 96, 96, 2)
X_val.shape: (1704, 96, 96, 2)
X_test.shape: (1704, 96, 96, 2)
6.890482 5.989441 5.7024393 -5.4777956 -5.0524907 -5.5057893


In [35]:
from tensorflow.keras.layers import Input, LeakyReLU, Dense, Activation, Flatten, Conv2D, Conv2DTranspose, MaxPooling2D, BatchNormalization, Reshape, Concatenate

In [40]:
model_AE.generate_AE_one_by_one()

Keeping latent dimension size for encoder same: 18
Latent dimension size for decoder of the 1th AE is: 18 = 18 * 1
Executing: encoder_18_sub1 = encoder(18)


NameError: name 'BatchNormalization' is not defined